# Experiment: Alternative approach to remove duplicate entries


## 1. Goal: Reduct the number of loops used to remove duplicate entries

The goal of the experiment is to attempt to **reduce the number of loops used to remove duplicate entries from a dataset**. The final output is a list of unique app entries. In other words, we are trying to identify unique apps and separating the duplicate entries

This is [data set](https://www.kaggle.com/lava18/google-play-store-apps) used for the analys. I contains data about approx. 10,000 Android apps from Google Play. The data was collected in August 2018.

First of all let's start by laying some assumptions.


### 1.1. Premises and assumptions

Considering my current stage of programming/data science training, I'll have some constraints as to the kind of objects and operations I can employ to solve the problem. Maybe there are even better ways to solve the problem, so I'd be interested in hearing your opinion about them.

Anyways, now we can start by exploring the data.

## 2. Data Exploration

In [1]:
# function to transform a csv file into a list of list
def csv_to_list(file_name):
    # opening the file
    opened_file = open(file_name,encoding='utf8')
    
    # transforming the file output in a lsit of lists
    from csv import reader
    read_file = reader(opened_file)
    data_set = list(read_file)
    return data_set

In [2]:
# transforming the csv files (datasets) into list of lists
apps_google = csv_to_list("googleplaystore.csv")

Once we have transformed the datasets into list of lists, we explore the headers of the datasets to ensure that we have all the relevant attributes (columns) needed to perform the analysis.

In [3]:
apps_google[0]

['App',
 'Category',
 'Rating',
 'Reviews',
 'Size',
 'Installs',
 'Type',
 'Price',
 'Content Rating',
 'Genres',
 'Last Updated',
 'Current Ver',
 'Android Ver']


------

In [4]:
# This function allows us to explore the number of columns, rows and datapoints
def explore_dataset(dataset):
    
    dataset_rows = len(dataset[1:])
    dataset_columns = len(dataset[0])
    
    number_of_datapoints = dataset_rows * dataset_columns
    message = "The dataset has {} columns, {:,} rows and {:,} datapoints".format(dataset_columns, dataset_rows, number_of_datapoints)
    print(message)

In [5]:
explore_dataset(apps_google)

The dataset has 13 columns, 10,841 rows and 140,933 datapoints


At this point we need to ensure that there are no wrong data and remove duplicates.

----

## 3. Remove inaccurate data

The first step is to verify that each row has the required amount of colums (13) based on the exploration we have done above

In [6]:
# isolating header and calculating its length, and separating the dataset
header = apps_google[0]
header_length = len(header)
dataset = apps_google[1:]

# isolating the apps (rows and index) that are different length compared to the header
different_length_rows = []

for row in enumerate(dataset):
    # counting the number of columns of the app
    app_details = row[1]
    number_of_columns = len(app_details)
    
    # attach the different rows to the different_length_rows list
    if number_of_columns != header_length:
        different_length_rows.append(row)

different_length_rows

[(10472,
  ['Life Made WI-Fi Touchscreen Photo Frame',
   '1.9',
   '19',
   '3.0M',
   '1,000+',
   'Free',
   '0',
   'Everyone',
   '',
   'February 11, 2018',
   '1.0.19',
   '4.0 and up'])]

The row 10,472 is missing a column. This means that is has to be removed from the dataset.

In [7]:
# extracting the index we need to target to remove contents
index_to_remove = different_length_rows[0][0]

# deleting the faulty entry from the dataset
del dataset[index_to_remove]

----

## 4. The Experiment: Removing duplicate entries

### 4.1. The Original Solution

The original solution can be found [here](https://github.com/dataquestio/solutions/blob/master/Mission350Solutions.ipynb).

The authors used the following process to arrive at the list of unique app entries.

**STEP 1: Separating unique and duplicate names**

With the original solution we create two separate lists containing the app names that are either unique (unique_apps) or duplicate (duplicate_apps).

![image.png](https://data-portfolio-images.s3.eu-central-1.amazonaws.com/Screenshot_2020-06-11+dataquestio+solutions.png)

And, we also have 9,659 unique names.

Loops over a list with 10,840 items.

----

**STEP 2: Create a dictionary with unique keys and, as value, the highest number of reviews**

![image.png](https://data-portfolio-images.s3.eu-central-1.amazonaws.com/Screenshot_2020-06-11+dataquestio+solutions(1).png)

Loops over a list with 10,840 items.

**STEP 3: Creating a list of unique entries**

![image.png](https://data-portfolio-images.s3.eu-central-1.amazonaws.com/Screenshot_2020-06-11+dataquestio+solutions(2).png)


Loops over a list with 10,840 items.

FINAL RESULT: To create the list of unique entries the authors have over 35,520 items.

----

### 4.2. My Experiment

My goal was to attempt to find a way to reduce the number of loops involved in the creation of the list of unique app entries with the highest number of reviews. The reason why we focus on the highest number of reviews is that, in oder to proced with the analysis, we must select a criteria to select one option for each set of duplicates - in order to avoid random selection of the entries.


#### Step 1 – Separate unique and duplicate entries

The thought process I had was to elaborate the data to create reusable parts (better inputs for the following process). So, the first thing I did was to the focus in my approach on both entries and names, and not only on the names. With this approach, I create two sets of input data:

In [8]:
# creating a list for duplicates
duplicate_apps = []

# creating a list for unique app names, and a clean list of lists with unique names
unique_apps = {}

# sorting the dataset
for app in dataset:
    app_name = app[0]
    if app_name in unique_apps:
        duplicate_apps.append(app)
    else:
        unique_apps[app_name] = app

# printing the number of duplicate entries
number_of_duplicate_apps = len(duplicate_apps)
print("There are {:,} duplicates".format(number_of_duplicate_apps))

There are 1,181 duplicates


* A list of all the duplicate entries
* A dictionary of unique names (the keys), and the first entry for the unique name (as the value). This will still allow me to use the in operator to search the keys but has the advantage of having the entry attached.


To better examine the situation, here is an example of a duplicate.

In [9]:
duplicate_app_name = duplicate_apps[0][0]

for app in dataset:
    app_name = app[0]
    if app_name == duplicate_app_name:
        print(app)

['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80805', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']
['Quick PDF Scanner + OCR FREE', 'BUSINESS', '4.2', '80804', 'Varies with device', '5,000,000+', 'Free', '0', 'Everyone', 'Business', 'February 26, 2018', 'Varies with device', '4.0.3 and up']


-----

In [10]:
print("There are {:,} unique names".format(len(unique_apps)))

There are 9,659 unique names


----

At this point we have

* 9,659 unique names
* 1,181 duplicate entries


#### Step 2 – updating the dictionary with duplicate entries that have a higher number of reviews

Because I have already isolated the data I need for this step in the previous one, I can focus the work only on the duplicate entries instead of the whole dataset.

In [11]:
# this loops update unique_apps entries if the duplicate has more reviews
for duplicate in duplicate_apps:
    
    # extracting the relevant information for the duplicate
    duplicate_name = duplicate[0]
    duplicate_reviews = int(duplicate[3])
    
    # extracting the reviews info about existing entry in unique_app dictionary
    unique_app_reviews = unique_apps[duplicate_name][3]
    unique_app_reviews = int(unique_app_reviews)
    
    # updating the entry if the duplicate has higher reviews
    if duplicate_reviews > unique_app_reviews:
        unique_apps[duplicate_name] = duplicate

We've looped over a list with 1,181 items.

At the end of this process we have an updated dictionary with unique keys and, as values, entries that have the highest number of ratings in the dataset.


#### Step 3 – creating a list of unique entries

To create the final list of unique values, we can loop over the dictionary and extract the entry for each unique key.

In [12]:
# crating the cleaned list of unique app entries
apps_google_duplicateless = []

for app_name in unique_apps:
        entry = unique_apps[app_name]
        apps_google_duplicateless.append(entry)

We've looped over a dictionary of 9,659 items and we've created the desired output.

## 5. Conclusion

By comparing the two approaches, we can see that the:

| Approach | Step 1 |  Step 2 | Step 3 | Step 4 |
|------|-----|-----|----|----|
| Original approach | 10,840 |  10,840 | 10,840 | 32,520 |
| Original approach | 10,840 |  1,191 | 9,659 | 21,680 |
 
SUMMARY:

* Loops saved: 10,840
* Improvement: 1,5x

This approach has saved an entire iteration over the whole dataset - it seems that I may have found a solution that is more efficient. I can't say at 100% because I haven't looked at speed or memory performance.

Loved running this experiemnt!

Thanks for reading. And, I you have some opinion, comments or feedback on this approach don't hesitate to reach out! Cheers!